# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [ ]:
import os
import tarfile
import urllib

: 

In [ ]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

: 

In [ ]:
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

: 

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

: 

In [ ]:
from matplotlib import pyplot
import matplotlib pyplot as plt

housing.hist(bins=50, figsize=(20, 15))
plt.show()

: 

In [ ]:
import numpy as np
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

: 

In [ ]:
from zlib import crc32
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

: 

In [ ]:
def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

: 

In [ ]:
housin_with_id = housing.reset_index()
train_set, test_set = split_train_test_by_id(housin_with_id, 0.2, "index")

: 

In [ ]:
housin_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_train_test_by_id(housin_with_id, 0.2, "id")

: 

In [ ]:
from skelearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
housing["income_cat"] = pd.cut(housing["median_income"], bins=[0.,1.5,3.0,4.5,6.,np.inf], labels=[1,2,3,4,5])

: 

In [ ]:
from sklearn.model_selection import StratifieldShuffleSplit

split = StratifieldShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for  train_index, test_index in split.split(housing, housing[income_cat]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

: 

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

: 

In [ ]:
housing = strat_train_set.copy()

: 

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")

: 

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

: 

In [ ]:
housing.ploy(kind="scatter", x="longitude", y="latitude", alpha=0.4),
s=housing["population"]/1000., label="population", figsize=(10, 7),
c="median_house_value", cmap=plt.get_cmp("jet"),colorbar=True,
)
plt.legend()

: 

In [ ]:
corr_matrix = housing.corr()

: 

In [ ]:
from pandas.plotting import scatter_matrix

: 

In [ ]:
attributes = ["median_house_value", "median_income", "total_rooms"],
housing_median_age
scatter_matrix(housing[attributes], figsize=(12, 6))

: 

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

: 

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["totalrooms"]\
housing["population_per_household"] = housing["population"]/housing["households"]

: 

In [ ]:
housin = strat_train_set.drop("median_house_value", axis=1)\
housing_labels = strat_train_set["median_house_value"].copy()

: 

In [ ]:
housing.dropna(subset=["total_bedrooms"])\
housing.drop("total_bedrooms, axis=1")\
median = housing["total_bedrooms"].median()\
housing["total_bedrooms"].fillana(median,inplace=True)

: 

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

: 

In [ ]:
housing_num housing.drop("ocean_proximity", axis=1)

: 

In [ ]:
imputer.fit(housing_num)

: 

In [ ]:
X = imputer.transform(housing_num)

: 

In [ ]:
    housing_tr = pd.DataFrame(X, colums=housing_num.columns, index=housing_num.index)

: 

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
room_ix, bedrooms_per_room = add_bedroom_per_room

: 

In [ ]:
def fit(self, X, y=None):
    return self

: 

In [ ]:
def transform(self, X, y=None):
    rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
    population_per_household = X[:, population_ix] / X[:, households_ix]
    if self.add_bedrooms_per_room:
        bedrooms_per_room = X[:, bedrooms_ix] / X[:,rooms_ix]
        return np.c_[X, rooms_per_household, population_per_household]
    
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

: 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

: 

In [ ]:
num_attribs = list(housing_num)
cat_attibs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attibs),
     
])

housing_prepared = full_pipeline.fit_transform(housing)

: 

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression
lin_reg.fit(housing_prepared, housing_labels)

: 

In [ ]:
some_data = housing, iloc[:5]
some_labels = housing labels.iloc[:5]
some_data_prepared = full_pipelin.transform(some_data)
print("Predications:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

: 

In [ ]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

: 

In [ ]:
from sklearn.tree import DecisionTreeRegressor 

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

: 

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_mse_rmse = np.sqrt(tree_mse)
tree_rmse

: 

Feito até pagina 58


In [ ]:
from skelearn.model_selection import cross_val_score
scores  = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv 10)
tree_rmse_scores = np.sqrt(-scores)

: 

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

: 

In [ ]:
import joblib
joblib.dump(my_model, "my_model.pkl")
my_model = joblib.load("my_model.pkl")

: 

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [3,10,30], 'max_features': [2,4,6,8]},
    ('bootstrap'): [False], 'n_estimators': [3,10], 'max_features': [2,3,4]
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

: 

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
Y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predications = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predications)
final_rmse = np.sqrt(final_mse)


: 

: 

: 

: 

: 

: 

: 